## agentを利用したアプリケーション

### 準備

必要なライブラリのインストール

In [ ]:
%pip install -r ../requirements.txt

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, CohereEmbeddings(model="embed-multilingual-v3.0"))
retriever = vector.as_retriever()

retriever.invoke("how to upload a dataset")[0]

USER_AGENT environment variable not set, consider setting it to identify your requests.
/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='grade over  evaluators=[exact_match], # The evaluators to score the results  experiment_prefix="sample-experiment", # The name of the experiment  metadata={    "version": "1.0.0",    "revision_id": "beta"  },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";const client = new Client();// Define dataset: these are your test casesconst datasetName = "Sample Dataset";const dataset = await client.createDataset(datasetName, {  description: "A sample dataset in LangSmith.

### Tool

In [2]:
from langchain_core.tools import tool
from enum import Enum

# 曜日のEnum定義
class DayOfWeek(Enum):
    MONDAY = 'Monday!'
    TUESDAY = 'Tuesday!'
    WEDNESDAY = 'Wednesday!'
    THURSDAY = 'Thursday!'
    FRIDAY = 'Friday!'
    SATURDAY = 'Saturday!'
    SUNDAY = 'Sunday!'

day_values_map = {
    DayOfWeek.MONDAY: 0,
    DayOfWeek.TUESDAY: 10,
    DayOfWeek.WEDNESDAY: 20,
    DayOfWeek.THURSDAY: 30,
    DayOfWeek.FRIDAY: 40,
    DayOfWeek.SATURDAY: 50,
    DayOfWeek.SUNDAY: 100,
}

@tool
def combat_power(day: DayOfWeek) -> int:
    """
    曜日に対応するBigBaByの戦闘力を計算します。

    Args:
      day: 曜日です。
    """

    return day_values_map.get(day)

print(combat_power.name)
print(combat_power.description)
print(combat_power.args)
print(combat_power)


combat_power
曜日に対応するBigBaByの戦闘力を計算します。

Args:
  day: 曜日です。
{'day': {'$ref': '#/$defs/DayOfWeek'}}
name='combat_power' description='曜日に対応するBigBaByの戦闘力を計算します。\n\nArgs:\n  day: 曜日です。' args_schema=<class 'langchain_core.utils.pydantic.combat_power'> func=<function combat_power at 0x795a0c9bbba0>


In [3]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
tools = [combat_power]#, retriever_tool]
tools

[StructuredTool(name='combat_power', description='曜日に対応するBigBaByの戦闘力を計算します。\n\nArgs:\n  day: 曜日です。', args_schema=<class 'langchain_core.utils.pydantic.combat_power'>, func=<function combat_power at 0x795a0c9bbba0>)]

In [4]:
from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r")
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke("おはよう")
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: おはようございます！今日も一日頑張りましょう。
ToolCalls: []


In [5]:
response = model_with_tools.invoke("BigBaByの月曜の次の日の戦闘力は？")
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: I will use the provided function to find out the combat power for the day after Monday, and relay this information to the user.
ToolCalls: [{'name': 'combat_power', 'args': {'day': 'Monday'}, 'id': '404b21b856f74d4ea834783268eac228', 'type': 'tool_call'}]


In [6]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver  # an in-memory checkpointer

system_message = "あなたは有用なアシスタントです。"
memory = MemorySaver()
langgraph_agent_executor = create_react_agent(
  model, tools,state_modifier=system_message, checkpointer=memory
)
config = {"configurable": {"thread_id": "test-thread"}}
langgraph_agent_executor.invoke(
  {
    "messages": [
      ("user","BigBaByの木曜の戦闘力は？"),
    ]
  },
  config,
)['messages']

[HumanMessage(content='BigBaByの木曜の戦闘力は？', additional_kwargs={}, response_metadata={}, id='30824695-7463-47f9-9f07-243c3766cc09'),
 AIMessage(content='I will use the provided function to find out the combat power for Thursday and relay this information to the user.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '5b6041ef-4f5f-4ffe-a1af-12733d37f484', 'tool_calls': [{'id': '2544a0e9c046432286ea32162c62832c', 'function': {'name': 'combat_power', 'arguments': '{"day": "Thursday"}'}, 'type': 'function'}], 'token_count': {'input_tokens': 925.0, 'output_tokens': 64.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '5b6041ef-4f5f-4ffe-a1af-12733d37f484', 'tool_calls': [{'id': '2544a0e9c046432286ea32162c62832c', 'function': {'name': 'combat_power', 'arguments': '{"day": "Thursday"}'}, 'ty